In [ ]:
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import numpy as np
import torch

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")

embedder = SentenceTransformer('all-MiniLM-L6-v2')

with open("Copy of Understanding Statistical Hypothesis Testing (OJT Notes).txt", "r", encoding="utf-8") as file:
    content = file.read()

chunks = [chunk.strip() for chunk in content.split("\n\n") if len(chunk.strip()) > 50]

chunk_embeddings = embedder.encode(chunks)

def answer_question(question):
    question_embedding = embedder.encode([question])[0]
    similarities = np.inner(chunk_embeddings, question_embedding)
    best_idx = int(np.argmax(similarities))
    best_context = chunks[best_idx]
    
    result = qa_model(question=question, context=best_context)
    return result["answer"]

gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Hypothesis Testing QA",
    description="Ask any question",
).launch()
